In [1]:
library(rdydisstools)
setwd('~/notebooks/dissertation')
loadpackages(c('doParallel', 'foreach', 'logging', 'glue', 'grid', 'gridExtra', 'dplyr', 'ggplot2', 'DescTools', 'careless'))
nDatasets <- 5000

In [2]:
registerDoParallel(72)
h2data <- foreach(i=1:nDatasets, .verbose=FALSE) %dopar% {
    # Initialize logging
    addHandler(writeToFile, file=glue::glue('~/notebooks/dissertation/logs/h2/sim{i}.log'), level='INFO')
    logstring <- glue::glue('dis.h2.{i}')
    
    # Specify path for sample
    datasetstring <- glue::glue("~/notebooks/dissertation/data/simsample{i}.RData")
    logging::loginfo(glue::glue('dataset loading from data/simsample{i}'), logger=logstring)
    
    #load sample dataset into memory
    load(datasetstring)
    logging::loginfo(glue::glue('dataframe has been loaded from: {datasetstring}'), logger=logstring)
    
    #Evaluate Hypothesis 1
    t(dispatchHypothesisEval(df, hypothesis="h2", lastColumn=100))
} %>% do.call("rbind.data.frame", .)

save(h2data, file='~/notebooks/dissertation/artifacts/h2/h2data.RData')

In [3]:
head(h2data)

hc_b,hc_lower,hc_upper,ht_b,ht_lower,ht_upper,hypTest,moreInf,hasOverlap,overlap
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.05432443,-0.1468308923,0.03818204,-0.04995976,-0.1434999,0.04358039,0,1,1,0.1816819
0.08011583,-0.0005948419,0.16082650,-0.03181818,-0.1238372,0.06020086,0,0,1,0.0607957
-0.06947368,-0.1633093459,0.02436198,-0.12948500,-0.2302141,-0.02875589,0,0,1,0.1345535
-0.03809524,-0.1291777537,0.05298728,-0.08462462,-0.1810540,0.01180476,0,0,1,0.1409825
-0.05560676,-0.1482257360,0.03701222,-0.05467707,-0.1481559,0.03880174,0,1,1,0.1851681
-0.04985015,-0.1419640465,0.04226374,-0.02549138,-0.1169462,0.06596342,0,1,1,0.1592099
